In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import Dataset

In [ ]:
df = pd.read_csv("D:\\data associate\project\\DataCoSupplyChainDataset.csv",encoding='latin1')
df.head()

In [ ]:
df.tail()

In [ ]:
df.loc[37]

In [ ]:
df.columns

# Columns selection

### See there are a lot of columns.
### But we don't require all the columns for our case study.
### We can drop columns related to customer details (first name, last name, email, password. etc.)
### Similarly we don't require sales per customer, delivery status,etc.
### We will drop all the unwanted columns.

In [ ]:
features = ['Days for shipping (real)',
            'Days for shipment (scheduled)',
            'Category Id',
            'Customer Segment',
            'Market',
            'Order Country',
            'Order Region',
            'order date (DateOrders)',
            'Order Item Quantity',
            'Order Status',
            'Product Card Id',
            'Product Price',
           'Shipping Mode']
len(features)

In [ ]:
df[features].head()

In [ ]:
df = df[features]

# EDA

In [ ]:
df.info()

In [ ]:
df.describe()

### We see that date has been cosidered as object datatype.
### So converting them into datetime datatype.

#### Will create new columns order day,date,month, order year from the order date coulmn.


In [ ]:
pd.to_datetime(df['order date (DateOrders)']).dt.date

In [ ]:
 pd.to_datetime(df['order date (DateOrders)']).dt.day

In [ ]:
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)']).dt.date
df['Order Day'] = pd.to_datetime(df['order date (DateOrders)']).dt.day
df['Order Date'] = pd.to_datetime(df['order date (DateOrders)']).dt.date
df['Order Month'] = pd.to_datetime(df['order date (DateOrders)']).dt.month
df['Order Year'] = pd.to_datetime(df['order date (DateOrders)']).dt.year

In [ ]:
df.head()

In [ ]:
df['Order Date']=pd.to_datetime(df['Order Date'])

#### Now we no loger require the order date(DateOrders)
#### So droping it.

In [ ]:
df.drop(columns=['order date (DateOrders)'],inplace=True)

### Check if there are any null values.

In [ ]:
df.isna().sum()

#### So there are no null values present.

## Plotting some graph to understand dataset.
## And EDA also

## Plot of product Price

In [ ]:
plt.title('Boxplot of Product Price')
sns.boxplot(x=df['Product Price'])

In [ ]:
plt.title('Distribustion of Product Price')
sns.distplot(df['Product Price'])

In [ ]:
Q1 = df['Product Price'].quantile(0.25)
Q3 = df['Product Price'].quantile(0.75)
IQR = Q3-Q1
LL = Q1-1.5*IQR
UL = Q3+1.5*IQR

In [ ]:
df[(df['Product Price']<LL) | (df['Product Price']>UL)]

In [ ]:
(2048/df.shape[0])*100

In [ ]:
df =df[(df['Product Price']>=LL) & (df['Product Price']<=UL)]

## Customer Segment Plots

In [ ]:
df['Customer Segment'].value_counts()

In [ ]:
plt.title('Share of Customer Segment')
df['Customer Segment'].value_counts().plot(kind = 'pie', autopct = '%.2f%%', cmap = 'Set2',explode = [0.05, 0.05, 0.05])

## Market Share Plot

In [ ]:
df['Market'].value_counts()

In [ ]:
plt.title('Market shares')
df['Market'].value_counts().plot(kind='pie',autopct = '%.2f%%', cmap = 'Set1',explode = [0.05, 0.05, 0.05,0.05,0.05])

In [ ]:
df['Order Year'].value_counts()

## Year wise quantity sold plot

In [ ]:
sns.countplot(data=df, x='Order Year')

In [ ]:
df[df['Order Year']==2018]

In [ ]:
df.drop(df[df['Order Year']==2018].index, axis=0, inplace=True)

In [ ]:
df.shape

## Plot for Regions

In [ ]:
df['Order Region'].value_counts()

In [ ]:
order = df['Order Region'].value_counts().index

In [ ]:
plt.title('Orders from different regions')
sns.countplot(data=df, y='Order Region',order=order, palette='rocket')

## Plot for Countries

In [ ]:
df['Order Country'].value_counts()[:20]

In [ ]:
plt.title('Top 20 Countries By Orders')
df['Order Country'].value_counts()[:20].sort_values(ascending=True).plot(kind='barh',color='darkviolet')
plt.xlabel('Total Order Quantity')

### PLot For Product Card Id

In [ ]:
df['Product Card Id'].value_counts()

In [ ]:
order = list(df['Product Card Id'].value_counts().sort_values()[:20].index)

In [ ]:
plt.title('Product car Id vs orders')
sns.countplot(data=df, y='Product Card Id',order=order, palette='crest')

### Different shipping modes plot

In [ ]:
plt.title('Number of orders for different shipping modes')
sns.countplot(data=df, x='Shipping Mode',palette='Set1')

### Order status plot

In [ ]:
plt.title('Order status')
sns.countplot(data=df, y='Order Status')

### Plot For Number Of Quantities Ordered

In [ ]:
df['Order Item Quantity'].value_counts()

In [ ]:
order = df['Order Item Quantity'].value_counts().index

In [ ]:
plt.title('Order Quantities')
sns.countplot(data=df, x='Order Item Quantity',order=order,palette='mako')
plt.ylabel('Quantity')
plt.show()

## Encoding

In [ ]:
df.dtypes

In [ ]:
cat_columns = []
for column in df.columns:
    if df[column].dtypes == 'object':
        cat_columns.append(column)
cat_columns

In [ ]:
new_df = pd.get_dummies(data=df, columns=cat_columns)

In [ ]:
for i in new_df.columns:
    print(i)

In [ ]:
new_df['Product Price']

In [ ]:
plt.figure(figsize=(50,50))
sns.heatmap(new_df.corr(numeric_only=True),cmap='coolwarm')

In [ ]:
df.columns

In [ ]:
df = df[['Category Id', 'Order Item Quantity','Product Card Id', 'Product Price','Order Date', 'Order Month', 'Order Year']]

# Heatmap

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True),annot=True,cmap='coolwarm')

In [ ]:
df.columns

In [ ]:
df.head()

## Data Cleaning

In [ ]:
plt.figure(figsize=(12, 8))
daily_orders= df.groupby(['Order Date'])['Order Item Quantity'].sum().reset_index()
sns.lineplot(data=daily_orders, x='Order Date', y='Order Item Quantity')
plt.title('Total Order Quantity Over Time')
plt.xlabel('Day')
plt.ylabel('Total Quantity')
plt.show()

In [ ]:
daily_orders.to_csv('Daily Orders.csv',index=False)

In [ ]:
sns.boxplot(data=daily_orders ,x='Order Item Quantity')

In [ ]:
Q1 = daily_orders['Order Item Quantity'].quantile(0.25)
Q3 = daily_orders['Order Item Quantity'].quantile(0.75)

IQR = Q3 - Q1

LL = Q1 - 1.5*IQR
UL = Q3 + 1.5*IQR

print(LL,UL)

In [ ]:
daily_orders[(daily_orders['Order Item Quantity']>=LL) & (daily_orders['Order Item Quantity']<=UL)]

In [ ]:
daily_orders.shape

In [ ]:
daily_orders = daily_orders[(daily_orders['Order Item Quantity']>=LL) & (daily_orders['Order Item Quantity']<=UL)]

In [ ]:
daily_orders

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=daily_orders, x='Order Date', y='Order Item Quantity')
plt.title('Total Order Quantity Over Time')
plt.xlabel('Day')
plt.ylabel('Total Quantity')
plt.show()

In [ ]:
sns.boxplot(data=daily_orders ,x='Order Item Quantity')

In [ ]:
cleaned_daily_orders = daily_orders.copy()
cleaned_daily_orders['Order Date'] = pd.to_datetime(cleaned_daily_orders['Order Date'])
cleaned_daily_orders['Year Month']=cleaned_daily_orders['Order Date'].dt.to_period('M')
cleaned_daily_orders['Year Week']=cleaned_daily_orders['Order Date'].dt.to_period('W')
cleaned_daily_orders

In [ ]:
cleaned_daily_orders.info()

In [ ]:
monthly_orders = cleaned_daily_orders.groupby('Year Month')['Order Item Quantity'].sum()

In [ ]:
plt.figure(figsize=(12, 6))
monthly_orders.plot(kind='line', marker='o')
plt.title('Total Orders by Month')
plt.ylabel('Total Order Quantity')
plt.xlabel('Month')
plt.grid(True)
plt.show()

In [ ]:
weekly_orders = cleaned_daily_orders.groupby('Year Week')['Order Item Quantity'].sum()

In [ ]:
sns.boxplot(x=weekly_orders)

In [ ]:
Q1 = weekly_orders.quantile(0.25)
Q3 = weekly_orders.quantile(0.75)

IQR = Q3-Q1

LL = Q1 - IQR*1.5
UL = Q3 + IQR*1.5

In [ ]:
 weekly_orders[(weekly_orders>=LL) & (weekly_orders<=UL)]

In [ ]:
weekly_orders.shape

In [ ]:
weekly_orders = weekly_orders[(weekly_orders>=LL) & (weekly_orders<=UL)]

In [ ]:
plt.figure(figsize=(12, 6))
weekly_orders.plot(kind='line', marker='o')
plt.title('Total Orders by Week')
plt.ylabel('Total Order Quantity')
plt.xlabel('Week')
plt.grid(True)
plt.show()

In [ ]:
df.head()

In [ ]:
cleaned_daily_orders

In [ ]:
cleaned_daily_orders.dtypes

In [ ]:
cleaned_daily_orders.to_csv('Cleaned Data.csv',index=False)

# Model Building

# **Prophet**

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from prophet.plot import plot_cross_validation_metric

In [ ]:
weekly_orders.head()

In [ ]:
weekly_orders_df = weekly_orders.reset_index()

In [ ]:
weekly_orders_df.dtypes

In [ ]:
weekly_orders_df.columns = ['ds','y']

In [ ]:
weekly_orders_df['ds'] = weekly_orders_df['ds'].dt.to_timestamp()
weekly_orders_df

In [ ]:
split_point = int(len(weekly_orders) * 0.80)
split_point

In [ ]:
train = weekly_orders_df.iloc[:split_point]
test = weekly_orders_df.iloc[split_point:]

In [ ]:
prophet_weekly_model = Prophet()
prophet_weekly_model.fit(train)


In [ ]:
weekly_future = prophet_weekly_model.make_future_dataframe(periods=len(test), freq='W-SUN')

In [ ]:
weekly_forecast = prophet_weekly_model.predict(weekly_future)

In [ ]:
# Evaluate on Training data
y_pred_train = weekly_forecast['yhat'][:split_point]
mae_train = mean_absolute_error(train['y'], y_pred_train)
mse_train = mean_squared_error(train['y'], y_pred_train)
rmse_train = np.sqrt(mse_train)

print(f"Training MAE: {mae_train}")
print(f"Training MSE: {mse_train}")
print(f"Training RMSE: {rmse_train}")

# Evaluate on Testing data
y_pred_test = weekly_forecast['yhat'][split_point:]
mae_test = mean_absolute_error(test['y'], y_pred_test)
mse_test = mean_squared_error(test['y'], y_pred_test)
rmse_test = np.sqrt(mse_test)

print(f"\nTesting MAE: {mae_test}")
print(f"Testing MSE: {mse_test}")
print(f"Testing RMSE: {rmse_test}")

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(train['ds'], train['y'], label='Training Data', color='blue')
plt.plot(test['ds'], test['y'], label='Actual Test Data', color='orange')
plt.plot(train['ds'], y_pred_train, label='Predicted Training Data', color='red', linestyle='--')
plt.plot(test['ds'], y_pred_test, label='Predicted Test Data', color='green', linestyle='--')
plt.title('Actual vs Predicted Weekly Orders')
plt.xlabel('Date')
plt.ylabel('Order Quantity')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
weekly_orders_df.shape

In [ ]:
df_cv_m1 = cross_validation(prophet_weekly_model, initial = '100 W', period= '1 W', horizon = '10 W')

In [ ]:
fig_rmse = plot_cross_validation_metric(df_cv_m1, metric='rmse')

In [ ]:
from prophet.diagnostics import performance_metrics
pm1=performance_metrics(df_cv_m1)

print("rmse:", pm1['rmse'])
#print("avg rmse:", pm1['rmse'].mean())

In [ ]:
pm1

In [ ]:
df_cv_m1


# **ARIMA AND SARIMA**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(weekly_orders_df['Order Item Quantity'],
                            model='multiplicative', period = 30)
fig = result.plot()
fig.set_size_inches(15,10)

In [ ]:
plt.figure(figsize=(20,10))
fig = pd.plotting.autocorrelation_plot(weekly_orders_df['Order Item Quantity'], )

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
plot_pacf(weekly_orders_df['Order Item Quantity'],lags=50)

In [ ]:
p, d ,q = 4, 0, 1

# **ARIMA**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
arima_model = ARIMA(weekly_orders, order=(p,d,q))
arima_model = arima_model.fit()
print(arima_model.summary())

In [ ]:
weekly_predictions = arima_model.predict(start=1)
weekly_predictions

In [ ]:
# weekly_predictions_test = arima_model.predict(start=split_point)
# weekly_predictions_test

In [ ]:
# Evaluate on Testing data
mae_test = mean_absolute_error(weekly_orders[split_point+1:],weekly_predictions[split_point:])
mse_test = mean_squared_error(weekly_orders[split_point+1:],weekly_predictions[split_point:])
rmse_test = np.sqrt(mse_test)

print(f"\nTesting MAE: {mae_test}")
print(f"Testing MSE: {mse_test}")
print(f"Testing RMSE: {rmse_test}")

In [ ]:
plt.figure(figsize=(15,8))
weekly_orders[:split_point].plot(kind='line',marker='o',color='blue')
weekly_orders[split_point:].plot(kind='line',marker='o',color='green')
weekly_predictions[:split_point].plot(kind='line',marker='o',color='red')
weekly_predictions[split_point:].plot(kind='line',marker='o',color='orange')

In [ ]:
def smape(actual, predicted):
    return 100* np.mean(2*np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted)))

In [ ]:
smape_arima = smape(weekly_orders[split_point:], weekly_predictions[split_point:])
smape_arima

# **SARIMA**

In [ ]:
import statsmodels.api as sm
import warnings

In [ ]:
p, d ,q = 4, 1, 1

In [ ]:
sarima_model = sm.tsa.statespace.SARIMAX(weekly_orders,
                                order=(p, d, q),
                                seasonal_order=(p, d, q, 52))
sarima_weekly_model=sarima_model.fit()
print(sarima_weekly_model.summary())

In [ ]:
sarima_pred = sarima_weekly_model.predict(start=1)

In [ ]:
# sarima_pred_test = sarima_weekly_model.predict(start=split_point+1,end=len(weekly_orders))

In [ ]:
sarima_pred

In [ ]:
# Evaluate on Testing data
mae_test = mean_absolute_error(weekly_orders[split_point+1:],sarima_pred[split_point:])
mse_test = mean_squared_error(weekly_orders[split_point+1:],sarima_pred[split_point:])
rmse_test = np.sqrt(mse_test)

print(f"\nTesting MAE: {mae_test}")
print(f"Testing MSE: {mse_test}")
print(f"Testing RMSE: {rmse_test}")

In [ ]:
plt.figure(figsize=(15,8))
weekly_orders[:split_point].plot(kind='line',color='blue')
weekly_orders[split_point:].plot(kind='line',color='green')
sarima_pred[:split_point].plot(kind='line',color='red',linestyle='--')
sarima_pred[split_point+1:].plot(kind='line',color='orange',linestyle='--')

In [ ]:
smape_arima = smape(weekly_orders[split_point:], sarima_pred[split_point:])
smape_arima

# **Linear** #

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
weekly_orders_df.columns = ['Week','Order Item Quantity']

In [ ]:
train = weekly_orders_df.iloc[:split_point]
test = weekly_orders_df.iloc[split_point:]

In [ ]:
weekly_orders_df.head()

In [ ]:
weekly_orders_df.dtypes

In [ ]:
X_train = train.drop(['Order Item Quantity'],axis=1).values.astype('float64')
y_train = train['Order Item Quantity'].values.astype('float64')

In [ ]:
X_test = test.drop(['Order Item Quantity'],axis=1).values.astype('float64')
y_test = test['Order Item Quantity'].values.astype('float64')

In [ ]:
linear_reg_model = LinearRegression()

In [ ]:
linear_reg_model.fit(X_train, y_train)

In [ ]:
linear_prediction_train = linear_reg_model.predict(X_train)

In [ ]:
linear_prediction_test = linear_reg_model.predict(X_test)

In [ ]:
# linear_prediction

In [ ]:

# Evaluate on Testing data
mae_test = mean_absolute_error(y_test,linear_prediction_test)
mse_test = mean_squared_error(y_test,linear_prediction_test)
rmse_test = np.sqrt(mse_test)

print(f"\nTesting MAE: {mae_test}")
print(f"Testing MSE: {mse_test}")
print(f"Testing RMSE: {rmse_test}")

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(train['Week'], train['Order Item Quantity'], label='Training Data', color='blue')
plt.plot(test['Week'], test['Order Item Quantity'], label='Actual Test Data', color='orange')
plt.plot(train['Week'], linear_prediction_train, label='Predicted Training Data', color='red', linestyle='--')
plt.plot(test['Week'], linear_prediction_test, label='Predicted Test Data', color='green', linestyle='--')
plt.title('Actual vs Predicted Weekly Orders')
plt.xlabel('Date')
plt.ylabel('Order Quantity')
plt.legend()
plt.grid(True)
plt.show()